In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

Using TensorFlow backend.


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
set_session(tf.Session(config=config))

In [3]:
ft = fastText.load_model("/home1/zishan/raghav/wiki.hi.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [4]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [5]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [6]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [7]:
def train_dev_sentences(filetrain, filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]
    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)

In [8]:
train_file = '/home1/zishan/raghav/Data/train_total.txt'
dev_file = '/home1/zishan/raghav/Data/test_total.txt'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(train_file, dev_file, 3)

In [9]:
n_words = 0
for sentence in train_sentences:
    n_words+=len(sentence)
print(n_words)

239628


In [10]:
from collections import Counter
print(Counter(train_labels))
print(labels2Idx)

Counter({0: 1475, 2: 368, 1: 212, 3: 176, 7: 147, 4: 143, 6: 62, 5: 52, 8: 32})
{'SADNESS': 0, 'NO-EMOTION': 1, 'SYMPATHY/PENSIVENESS': 2, 'OPTIMISM': 3, 'JOY': 4, 'ANGER': 5, 'DISGUST': 6, 'FEAR/ANXIETY': 7, 'SURPRISE': 8}


In [11]:
print(train_sentences[:2])
print(train_labels[:2])
print(labels2Idx)
print(len(train_labels))

['इसके बाद से यहां पर भारतीय सेना तैनात रहती है।', 'सरकारी मीडिया ने सोमवार को बताया कि बाढ़ के भंवर में फंस कर फुजिआन प्रांत में ४३, हुनान में ७८ और गुआंगडांग में ३३ लोगों की मौत हो गई।']
[0, 0]
{'SADNESS': 0, 'NO-EMOTION': 1, 'SYMPATHY/PENSIVENESS': 2, 'OPTIMISM': 3, 'JOY': 4, 'ANGER': 5, 'DISGUST': 6, 'FEAR/ANXIETY': 7, 'SURPRISE': 8}
2667


In [12]:
n_labels = len(labels2Idx)

In [13]:
def compile_model_bilstm_cnn(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

# Build baseline Model

In [18]:
train_file = '/home1/zishan/raghav/Data/train_total.txt'
batch_size = 16
check_for_generator = 3
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [ ]:
for test in range(5):
    model = compile_model_bilstm_cnn(no_labels = n_labels)
    weights_file =f"/home1/zishan/raghav/weights/without_pretraining_{test}.h5"
    log_file = f"/home1/zishan/raghav/logs/without_pretraining_{test}.txt"
    print(weights_file, log_file)
    
    f = open(log_file, 'w', encoding='utf-8')
    f.close()

    max_f1 = 0
    for epoch in range(100):
        print("Epoch {}".format(epoch))
        model.fit_generator(sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                                 labels2Idx= labels2Idx,tokenize= tokenize),
                            steps_per_epoch= steps_per_epoch, epochs=1,)

        testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
        for i in range(len(dev_sentences)):
            testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
        results = model.predict(testset_features)


        predLabels = results.argmax(axis=-1)
        devLabels = dev_labels
        f1 = f1_score(devLabels, predLabels, average='macro') # offensive is the major class. So other is minor
        r = recall_score(devLabels, predLabels, average='macro')
        p = precision_score(devLabels, predLabels, average='macro')
        a = accuracy_score(devLabels, predLabels)

        if f1> max_f1:
            print("Saved ")
            model.save_weights(weights_file)
            with open(log_file,'a+') as f:
                text = "{0} , a:{1}, f1:{2}, p:{3}, r:{4}\n".format(epoch, a, f1, p, r)
                f.write(text)
            max_f1 = f1

        print(a,f1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 75, 200)      320800      input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_6 (LeakyReLU)       (None, 75, 200)      0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 73, 200)      120200      leaky_re_lu_6[0][0]              
__________________________________________________________________________________________________
conv1d_5 (

/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Saved 
0.5598802395209581 0.0797611431008744
Epoch 1
Epoch 1/1
167/167 [==============================] - 41s 246ms/step - loss: 1.4552 - acc: 0.5580
Saved 
0.5748502994011976 0.11528244113609966
Epoch 2
Epoch 1/1
167/167 [==============================] - 41s 246ms/step - loss: 1.4099 - acc: 0.5618
Saved 
0.5703592814371258 0.1336253356707268
Epoch 3
Epoch 1/1
167/167 [==============================] - 41s 245ms/step - loss: 1.3492 - acc: 0.5715
Saved 
0.5778443113772455 0.21186482612516203
Epoch 4
Epoch 1/1
167/167 [==============================] - 40s 242ms/step - loss: 1.2749 - acc: 0.5812
Saved 
0.6002994011976048 0.23037089031534663
Epoch 5
Epoch 1/1
167/167 [==============================] - 40s 242ms/step - loss: 1.1983 - acc: 0.5939
Saved 
0.6032934131736527 0.2393369106449409
Epoch 6
Epoch 1/1
167/167 [==============================] - 41s 245ms/step - loss: 1.1055 - acc: 0.6186
Saved 
0.6062874251497006 0.27808646764573164
Epoch 7
Epoch 1/1
 64/167 [==========>.............